Class Balance in Training set - Experiment 3 - distilBERT

In [ ]:
R_SEED = 123

import transformers
from transformers import set_seed
set_seed(R_SEED)

import random
random.seed(R_SEED)

import numpy as np
np.random.seed(R_SEED)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.manual_seed(R_SEED)
torch.cuda.manual_seed(R_SEED)
torch.cuda.manual_seed_all(R_SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

cuda


In [ ]:
#Import all required packages
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score

Get the dataset from Kaggle (a CSV file) ans plit the data intro training - dev - test.

In [ ]:
!gdown 17-D6ZC8k2XFh3XbjMkbeoSVSsf8vGie6

Downloading...
From: https://drive.google.com/uc?id=17-D6ZC8k2XFh3XbjMkbeoSVSsf8vGie6
To: /content/Combined Data.csv
100% 31.5M/31.5M [00:00<00:00, 71.3MB/s]


In [ ]:
!head -n 5 Combined\ Data.csv

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless heart. All out of tune",Anxiety
2,"All wrong, back off dear, forward doubt. Stay in a restless and restless place",Anxiety
3,I've shifted my focus to something else but I'm still worried,Anxiety


Get rid of NAN lines.

In [ ]:
data = pd.read_csv('/content/Combined Data.csv').dropna()
X = data['statement']
y = data['status']

Mapping all the labels with different numbers (from 0 to 6 as there are 7 different labels)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
enc = preprocessing.LabelEncoder()
encoded_y = enc.fit_transform(y)
print(list(enc.classes_))
# decoded_y = enc.inverse_transform(encoded_y)

['Anxiety', 'Bipolar', 'Depression', 'Normal', 'Personality disorder', 'Stress', 'Suicidal']


In [ ]:
dev_pcrt = 0.2

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(
  X,
  encoded_y,
  test_size=dev_pcrt,
  random_state=R_SEED,
  shuffle=True,
  stratify=encoded_y,
)

X_train, X_dev, y_train, y_dev = train_test_split(
  X_train,
  y_train,
  test_size=dev_pcrt,
  random_state=R_SEED,
  shuffle=True,
  stratify=y_train,
)

#Class balance only in the training set
train_max_per_class = len(X_train) // len(list(enc.classes_))
train_df = pd.DataFrame({"X": list(X_train), "y": y_train})
train_filtered_df = train_df.groupby("y").apply(lambda x: x.sample(min(len(x), train_max_per_class), random_state=R_SEED)).reset_index(drop=True)
X_train = np.array(train_filtered_df["X"].tolist())
y_train = train_filtered_df["y"].values

for label in range(0, 7):
    print(f"Label {label}: {train_filtered_df['y'].value_counts().get(label, 0)}")


Label 0: 2458
Label 1: 1777
Label 2: 4816
Label 3: 4816
Label 4: 690
Label 5: 1656
Label 6: 4816


<ipython-input-22-9c11e11c2fd6>:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_filtered_df = train_df.groupby("y").apply(lambda x: x.sample(min(len(x), train_max_per_class), random_state=R_SEED)).reset_index(drop=True)


In [ ]:
del encoded_y, R_SEED, X, data, dev_pcrt, y

Initilalize the 3 necessary functions:
* `__init__`: This is executed whenever an instance of this is created.
* `__getitem__`: This allows you to access any element using an index or index list.
* `__len__`: This returns the number of elements.

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X_data, y_data, tokenizer):
      super(Dataset, self).__init__()
      self.data = []
      self.tokenizer = tokenizer
      for x, y in zip(X_data, y_data):
        x = self.tokenizer(x, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        x = { k:v.squeeze(0) for k,v in x.items() }
        self.data.append([x, y])

  def __getitem__(self, idx):
      sent = self.data[idx][0]
      lbl = self.data[idx][1]
      return sent, lbl

  def __len__(self):
      return len(self.data)

def collate_fn(batch):
    inputs, labels = zip(*batch)
    # Convert list of dicts into dict of tensors
    batch_inputs = {key: torch.stack([x[key] for x in inputs]) for key in inputs[0].keys()}
    # Convert labels into a tensor
    batch_labels = torch.tensor(labels, dtype=torch.long)
    return batch_inputs, batch_labels

Upload the BERT model. In our case, distilBERT architecture.

In [ ]:
# https://huggingface.co/distilbert/distilbert-base-cased
MODEL_NAME = 'distilbert/distilbert-base-cased'
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
train_set = Dataset(X_train, y_train, tokenizer)
dev_set = Dataset(X_dev, y_dev, tokenizer)
test_set = Dataset(X_test, y_test, tokenizer)

In [ ]:
del X_dev, X_test, X_train, tokenizer, y_dev, y_train, y_test

EXPERIMENT 3 - EXPERIMENTAL SETTING + WEIGHT DECAY + SCHEDULER

In [ ]:
#Hyperparameters
BATCH_SIZE = 64
LR = 5e-5
EPOCHS = 5
WEIGHT_DECAY = 1e-2

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE,
                                           shuffle=True, collate_fn=collate_fn)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, batch_size=BATCH_SIZE,
                                         shuffle=False, collate_fn=collate_fn)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE,
                                          shuffle=False, collate_fn=collate_fn)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device, scheduler):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_acc = 0, 0
    all_preds, all_labels = [], []

    for batch, (X, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
      X = {key: tensor.to(device) for key, tensor in X.items()}
      y = y.to(device)
      # Compute prediction and loss
      pred = model(**X).logits
      loss = loss_fn(pred, y)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Store loss and accuracy
      train_loss += loss.item()
      preds = pred.argmax(1)
      train_acc += (preds == y).type(torch.float).sum().item()

      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(y.cpu().numpy())

      if batch % 50 == 0:
        loss, current = loss.item(), batch * len(y)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    f1 = f1_score(all_labels, all_preds, average="macro")

    scheduler.step() # Calculate the new learning rate for the next epoch
    train_loss /= num_batches
    train_acc /= size
    return train_loss, train_acc, f1

Training and dev evaluation loops.

In [ ]:
def eval_loop(dataloader, model, loss_fn, device):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, test_acc = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X, y in tqdm(dataloader):
          X = {key: tensor.to(device) for key, tensor in X.items()}
          y = y.to(device)
          pred = model(**X).logits
          test_loss += loss_fn(pred, y).item()
          preds = pred.argmax(1)
          test_acc += (preds == y).type(torch.float).sum().item()
          all_preds.extend(preds.cpu().numpy())
          all_labels.extend(y.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average="macro")

    test_loss /= num_batches
    test_acc /= size
    print(f"Eval Error: \n F1-macro: {f1} Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, test_acc, f1

In [ ]:
best_dev_f1 = 0.0

model = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(list(enc.classes_))).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
loss_fn = torch.nn.CrossEntropyLoss()

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_acc, train_f1 = train_loop(train_loader, model, loss_fn, optimizer, device, scheduler)
    print(f'train loss: {train_loss}, train_acc: {train_acc}, train_f1-macro: {train_f1}')
    dev_loss, dev_acc, dev_f1 = eval_loop(dev_loader, model, loss_fn, device)
    print(f'dev loss: {dev_loss}, dev_acc: {dev_acc}, dev_f1-macro: {dev_f1}')
    if dev_f1 > best_dev_f1: #If current dev_acc is greater than the best value so far
        best_dev_f1 = dev_f1 # store it
        torch.save(model.state_dict(), 'model.pth')  # save the model to disk under the name of model.pth

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1
-------------------------------


  0%|          | 1/329 [00:03<19:42,  3.61s/it]

loss: 1.955739  [    0/21029]


...

loss: 0.115504  [19200/21029]


100%|██████████| 329/329 [15:43<00:00,  2.87s/it]


train loss: 0.10214090670563711, train_acc: 0.9666650815540444, train_f1-macro: 0.9720584739788466


100%|██████████| 132/132 [02:10<00:00,  1.01it/s]


Eval Error: 
 F1-macro: 0.8106873491967906 Accuracy: 83.0%, Avg loss: 0.538846 

dev loss: 0.5388461945183349, dev_acc: 0.8296357812314628, dev_f1-macro: 0.8106873491967906


Evaluate now in the test set.

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(list(enc.classes_))).to(device)
state_dict = torch.load("model.pth")
model.load_state_dict(state_dict)
model = model.to(device)
test_loss, test_acc, test_f1 = eval_loop(test_loader, model, loss_fn, device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-32-cfbb2a22d514>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowl

Eval Error: 
 F1-macro: 0.7992194086909281 Accuracy: 82.0%, Avg loss: 0.572918 

